In [1]:
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [2]:
model = "keepitreal/vietnamese-sbert"
embeddings = HuggingFaceEmbeddings(model_name = 'VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')

d:\Hust Study\Intern\Training\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
No sentence-transformers model found with name VoVanPhuc/sup-SimCSE-VietNamese-phobert-base. Creating a new one with mean pooling.
d:\Hust Study\Intern\Training\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
loader = PyPDFLoader('Data/Rule.pdf')
document = loader.load()
splitter = RecursiveCharacterTextSplitter(
        chunk_size=500, 
        chunk_overlap=50,
        )
chunks = splitter.split_documents(document)
vector_store = FAISS.from_documents(chunks, embedding=embeddings)
vector_store.save_local("vectordb")


Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 88 0 (offset 0)


In [6]:
vectorstore = FAISS.load_local("vectordb", embeddings, allow_dangerous_deserialization=True)

question = "nhân viên làm việc mấy giờ một ngày"
docs = vectorstore.similarity_search(question,k=3)
print(docs)

[Document(page_content='ARROW TECHNOLOGIES VIETNAM CO., LTD Room 02, Block A, 18th Floor, VTC Online Building, 18 Tam Trinh Str., Minh Khai Ward, Hai Ba Trung Dist., Hanoi Tel: 04.36331742\t\r \xa0\t\r \xa0NỘI QUY CÔNG TY Dành cho: Thực tập sinh 1. Quy định về thời gian thực tập • Thời gian làm việc của toàn công ty: từ 8: 30 đến 12h00 và từ 13:00 đến 17: 30 hàng ngày từ thứ Hai đến thứ Sáu.  • Thời gian thực tập tối thiểu của 1 thực tập sinh phần mềm là 20 giờ/ tuần.  • Trước khi bắt đầu kỳ thực tập, thực tập sinh đăng ký lịch', metadata={'source': 'Data/Rule.pdf', 'page': 0}), Document(page_content='gian làm việc của công ty với lý do không hợp lý đều sẽ ảnh hướng đến kết quả đánh giá thực tập sinh vào cuối đợt thực tập.  2. Một số quy định chung trong giờ làm việc • Ăn mặc gọn gàng, lịch sự, phù hợp môi trường văn phòng. • Không làm việc riêng trong giờ làm việc (xem phim, chat, Facebook … những nội dung không phục vụ cho công việc đang làm).. • Không download dữ liệu cá nhân (phim,

In [8]:
import os
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
 
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  }
]

model = genai.GenerativeModel(model_name='gemini-pro', safety_settings=safety_settings, )

prompt_parts = [
  '''
Input: Greece is larger than mexico.
Knowledge: Greece is approximately 131,957 sq km, while Mexico is approximately 1,964,375 sq km, making Mexico 1,389% larger than Greece.

Input: Glasses always fog up.
Knowledge: Condensation occurs on eyeglass lenses when water vapor from your sweat, breath, and ambient humidity lands on a cold surface, cools, and then changes into tiny drops of liquid, forming a film that you see as fog. Your lenses will be relatively cool compared to your breath, especially when the outside air is cold.

Input: A fish is capable of thinking.
Knowledge: Fish are more intelligent than they appear. In many areas, such as memory, their cognitive powers match or exceed those of ’higher’ vertebrates including non-human primates. Fish’s long-term memories help them keep track of complex social relationships.

Input: A common effect of smoking lots of cigarettes in one’s lifetime is a higher than normal chance of getting lung cancer.
Knowledge: Those who consistently averaged less than one cigarette per day over their lifetime had nine times the risk of dying from lung cancer than never smokers. Among people who smoked between one and 10 cigarettes per day, the risk of dying from lung cancer was nearly 12 times higher than that of never smokers.

Input: A rock is the same size as a pebble.
Knowledge: A pebble is a clast of rock with a particle size of 4 to 64 millimetres based on the Udden-Wentworth scale of sedimentology. Pebbles are generally considered larger than granules (2 to 4 millimetres diameter) and smaller than cobbles (64 to 256 millimetres diameter).

Input: Part of golf is trying to get a higher point total than others.
Knowledge:
''', 
]
respone = model.generate_content(prompt_parts)
print(respone.text) 

The object of the game is to complete 18 holes using the fewest strokes possible. The lower the number of strokes, the better.


In [7]:
def format_docs(docs):
    return "\n".join(doc.page_content for doc in docs)

def gennerate(question):
    retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 5})

    docs = retriever.get_relevant_documents(question)
    context = format_docs(docs)
    prompt = f"""
    Bạn là trợ lý cho các nhiệm vụ trả lời câu hỏi, hãy trả lời bằng tiếng Việt, lịch sự và thân thiện.
    Hãy trả lời câu hỏi dựa trên dữ liệu có trong đoạn ngữ cảnh
    Hãy trả lời không biết nêu không có thông tin trong đoạn ngữ cảnh

    Đoạn ngữ cảnh: {context}

    Câu hỏi: {question}
    """
    return model.generate_content(prompt).text

print(gennerate("Nhân viên làm việc mấy giờ một ngày"))

Từ 8 giờ 30 phút đến 12 giờ 00 và từ 13 giờ 00 đến 17 giờ 30


In [8]:
import gradio as gr
import random
import time

with gr.Blocks() as demo: 
    chatbot = gr.Chatbot(height=700)
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        bot_message = gennerate(message)
        chat_history.append((message, bot_message))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
